In [1]:
import pandas as pd
import datetime as datetime

In [8]:
# load predictions
def get_predictions(file_path):
    try:
        df = pd.read_csv(file_path, delimiter=';')
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.sort_values(by='datetime')
        print('predictions loaded')
        return df

    except Exception as e:
        print(f"Error reading and storing CSV: {e}")
        return None

# data_frame = get_predictions('../data/predictions/mock_predictions.csv')
# print(data_frame)
    

In [9]:
def get_energy_consumption_for_distance(distance_in_km):
    return 23.9 * distance_in_km / 100

def get_schedule(weekly_schedule, data_frame):
    if weekly_schedule is None:
        default_schedule = pd.read_csv('../data/schedules/default_schedule.csv', delimiter=';')
        default_schedule['datetime'] = pd.to_datetime(default_schedule['datetime'])
        default_schedule = default_schedule.sort_values(by='datetime')
        default_schedule['ev_energy_consumption_kWh'] = 0.0
        
        # created adjusted daytime for default_schedule
        offset = data_frame['datetime'].min().date() - default_schedule['datetime'].min().date()
        default_schedule['datetime'] = default_schedule['datetime'] + offset
        ## update ev_energy_consumption based on trips
        for index, row in default_schedule.iterrows():
            default_schedule.loc[default_schedule['datetime'] == row['datetime'], 'ev_energy_consumption_kWh'] = get_energy_consumption_for_distance(row['distance_in_km'])
        print('default schedule loaded')

        return default_schedule
    else:
        print('weekly schedule loaded ', weekly_schedule)
        return weekly_schedule

def add_schedule_to_df(weekly_schedule, data_frame):
    data_frame['ev_energy_consumption_kWh'] = 0.0
    for index, row in weekly_schedule.iterrows():
        data_frame.loc[data_frame['datetime'] == row['datetime'], 'ev_energy_consumption_kWh'] = get_energy_consumption_for_distance(row['distance_in_km'])

    return 
    
# weekly_schedule = get_schedule(None, data_frame)
# add_schedule_to_df(weekly_schedule, data_frame)
# print(data_frame)

In [11]:
# calculate remaining battery level per hour
# based on that delete all rows where battery level is below 20% so that they cannot be suggested
def calculate_battery_level_per_day(dataframe, battery_capacity, initial_battery_level):
    df_result = dataframe.copy()
    df_result['batteryLevelInPercent'] = 0.0
    
    previous_battery_level = initial_battery_level*100
    # Calculate the net change in battery level based on elapsed time
    for index, row in df_result.iterrows():
        net_percentage_change = row['ev_energy_consumption_kWh'] *  100 / battery_capacity
        current_battery_level = max(previous_battery_level - net_percentage_change, 0)
        df_result.at[index, 'batteryLevelInPercent'] = current_battery_level 
        
        # Feature: don't recommend charging while driving by dropping this entry
        if row['ev_energy_consumption_kWh'] > 0:
            print('not considering charging while driving at: ' + str(row['datetime']))
            df_result.drop(index, inplace=True)
        
        previous_battery_level = current_battery_level
    
    print('battery level calculated')
    return df_result

def delete_rows_with_low_battery(dataframe, battery_level):
    df_result = dataframe.copy()
    df_above_battery_level = df_result[df_result['batteryLevelInPercent'] >= battery_level]
    df_below_battery_level = df_result[df_result['batteryLevelInPercent'] < battery_level]
    
    if len(df_below_battery_level) > 0:
        min_row = df_below_battery_level[df_below_battery_level['datetime'] == df_below_battery_level['datetime'].min()].iloc[0]
        print('have to charge before battery too low (' + str(min_row['batteryLevelInPercent']) + ') at ' + str(min_row['datetime']))
    return df_above_battery_level

# data_frame = calculate_battery_level_per_day(data_frame, batteryCapacityInKWh, batteryLevel)
# data_frame = delete_rows_with_low_battery(data_frame, 20)
# print(data_frame)

In [12]:
# calculate relative charging emissions for each hour
emission_factors = {
    'Coal—PC': 820,
    'Gas—Combined Cycle': 490,
    'Biomass—cofiring': 740,
    'Biomass—dedicated': 230,
    'Geothermal': 38,
    'Hydropower': 24,
    'Nuclear': 12,
    'Concentrated Solar Power': 27,
    'Solar PV—rooftop': 41,
    'Solar PV—utility': 48,
    'Wind onshore': 11,
    'Wind offshore': 12
}

def calculate_total_emissions(data_frame, emission_factors):
    df_result = data_frame.copy()

    # Calculate total emissions for each row
    df_result['TotalEmissions'] = df_result.apply(
        lambda row: sum(row[column] * emission_factors[column] for column in df_result.columns[1:-4]),axis=1
    )
    
    print('total emissions calculated')
    return df_result

#data_frame = calculate_total_emissions(data_frame, emission_factors)
# print(data_frame)

In [13]:
# calculate charging times for starting at each hour
charging_curve = [204, 207, 212, 212, 212, 213, 214, 215, 215, 215, 216, 217, 217, 217, 218, 218, 219, 219, 220, 220, 220, 221, 221, 221, 221, 222, 222, 222, 214, 214, 215, 215, 171, 171, 171, 172, 172, 172, 172, 172, 172, 172, 173, 173, 173, 173, 173, 173, 174, 174, 174, 175, 175, 175, 175, 176, 175, 160, 160, 160, 150, 150, 151, 143, 144, 144, 145, 135, 136, 136, 128, 129, 128, 100, 101, 101, 101, 101, 102, 74, 69, 60, 51, 43, 35, 30, 28, 25, 29, 39, 39, 39, 39, 39, 39, 37, 35, 34, 32, 30]

def get_time_to_charge(
    battery_capacity: float,
    charging_curve: list[float],
    current_soc: int,
    target_soc: int = 80,
    efficiency: float = 0.9,
) -> float:
    """
    Get the estimated time to charge from the current state of charge (SOC) to a target SOC.

    Args:
        battery_capacity: The capacity of the battery
        charging_curve: The charging curve of the battery
        current_soc: The current state of charge (between 0 and 100)
        target_soc: The target state of charge (between 0 and 100, defaults to 80)
        efficiency: The efficiency of the charging process (between 0 and 1, defaults to 0.9)

    Returns:
        The estimated time to charge in h
    """

    one_percent_capacity = battery_capacity / 100
    charging_rates = zip(
        charging_curve[current_soc:target_soc],
        charging_curve[current_soc + 1 : target_soc + 1],
    )
    return (
        sum(
            2 * one_percent_capacity / (current_rate + next_rate)
            for (current_rate, next_rate) in charging_rates
        )
        / efficiency
    )

def calculate_charging_time(data_frame, battery_capacity, charging_curve):
    df_result = data_frame.copy()
    df_result['charging_duration'] = 0
    for index, row in df_result.iterrows():
        df_result.at[index, 'charging_duration'] = int(get_time_to_charge(battery_capacity, charging_curve, int(row['batteryLevelInPercent'])))
    print('charging times calculated')
    return df_result

# data_frame = calculate_charging_time(data_frame, batteryCapacityInKWh, charging_curve)
# print(data_frame)

In [14]:
def calculate_sum_emissions(data_frame): 
    df_result = data_frame.copy()

    df_result['SumEmissionsForBeginChargingNow'] = 0.0
    
    # Calculate the sum of emissions for each datetime along with the emissions of the following x hours
    for index, row in df_result.iterrows():
        hours_to_sum = row['charging_duration']  # Sum emissions for up to the next 6 hours
        sum_emissions = row['TotalEmissions']  # Use the TotalEmissions value
    
        # Add emissions of the following x hours
        for i in range(0, hours_to_sum):
            next_hour_index = index + i
            if next_hour_index < len(df_result):
                sum_emissions += df_result.at[next_hour_index, 'TotalEmissions']
                
        df_result.at[index, 'SumEmissionsForBeginChargingNow'] = sum_emissions
    print('sum emissions calculated')
    return df_result

# data_frame = calculate_sum_emissions(data_frame)
# print(data_frame)

In [15]:
def print_output(data_frame):    
    # output the top three charging times (beginning and ending) with the lowest emissions
    # sort by sum of emissions
    data_frame = data_frame.sort_values(by='SumEmissionsForBeginChargingNow')
    data_frame['charging_finished'] = data_frame['datetime'] + pd.to_timedelta(data_frame['charging_duration'], unit='h')
    print("According to the model, the best times to charge are:")
    print(data_frame[['datetime', 'charging_finished', 'charging_duration', 'SumEmissionsForBeginChargingNow']][:3])

In [16]:
## All in one function
def charging_scheduler (predictions_path, battery_capacity, battery_level, maximum_charging_power, weekly_schedule):
    
    predictions = get_predictions(predictions_path)
    weekly_schedule = get_schedule(None, predictions)
    add_schedule_to_df(weekly_schedule, predictions)
    predictions = calculate_battery_level_per_day(predictions, battery_capacity, battery_level)
    predictions = delete_rows_with_low_battery(predictions, 20)
    predictions = calculate_total_emissions(predictions, emission_factors)
    predictions = calculate_charging_time(predictions, battery_capacity, charging_curve)
    predictions = calculate_sum_emissions(predictions)
    print_output(predictions)

In [17]:
# define values
batteryCapacityInKWh = 70 
batteryLevel= 1 # 1 = 100%
maximumChargingPowerInKW = 7.6
file_path = '../data/predictions/mock_predictions.csv'

charging_scheduler(file_path, batteryCapacityInKWh, batteryLevel, maximumChargingPowerInKW, None)

predictions loaded
default schedule loaded
not considering charging while driving at: 2023-11-28 08:00:00
not considering charging while driving at: 2023-11-28 18:00:00
not considering charging while driving at: 2023-11-29 08:00:00
not considering charging while driving at: 2023-11-29 18:00:00
not considering charging while driving at: 2023-11-30 08:00:00
not considering charging while driving at: 2023-11-30 18:00:00
not considering charging while driving at: 2023-12-01 08:00:00
not considering charging while driving at: 2023-12-01 17:00:00
not considering charging while driving at: 2023-12-02 08:00:00
not considering charging while driving at: 2023-12-02 16:00:00
not considering charging while driving at: 2023-12-02 18:00:00
not considering charging while driving at: 2023-12-02 19:00:00
not considering charging while driving at: 2023-12-02 20:00:00
not considering charging while driving at: 2023-12-03 15:00:00
not considering charging while driving at: 2023-12-03 16:00:00
not consider